# This is the Jupyter Notebook for the MAGICODE project - TEST

In [ ]:
import os
os.chdir('datasets')

Unzip the dataset

In [ ]:
# join the volumes together into a single zip file
!zip -s 0 dataset.zip -O dataset_joined.zip
# unzip the newly assembled archive
!unzip dataset_joined.zip -d ./all_data
print ('files unzipped')

Split dataset into training and evaluation sets

In [ ]:
import sklearn.model_selection as model_selection
import glob

# define source folder
source = 'all_data/dataset'
# get all file paths
all_files = os.listdir(source)
# build a generic image path (e.g. 'all_data/dataset/*.png')
images_path = os.path.join(source, '*.png')
# get all images paths
img_files = glob.glob(images_path)

# training set will be 6 times the size of the evaluation set
distribution=6
TRAINING_SET_NAME = "training_set"
EVALUATION_SET_NAME = "eval_set"

# splits randomly the files into two sets
train_set,eval_set = train_test_split(img_files, train_size=(distribution / 10))

# copy the files (img and gui) from the all_data folder into the training_set folder
for file in train_set:
    shutil.copyfile("{}/{}.png".format(source, file), "{}/{}/{}.png".format(os.path.dirname(source), TRAINING_SET_NAME, file))
    shutil.copyfile("{}/{}.gui".format(source, file), "{}/{}/{}.png".format(os.path.dirname(source), TRAINING_SET_NAME, file))

# copy the files (img and gui) from the all_data folder into the eval_set folder
for file in eval_set:
    shutil.copyfile("{}/{}.png".format(source, file), "{}/{}/{}.png".format(os.path.dirname(source), EVALUATION_SET_NAME, file))
    shutil.copyfile("{}/{}.gui".format(source, file), "{}/{}/{}.png".format(os.path.dirname(source), EVALUATION_SET_NAME, file))

print("Training dataset: {}/training_set".format(os.path.dirname(source), path))
print("Evaluation dataset: {}/eval_set".format(os.path.dirname(source), path))


Transoform training set into numpy arrays

In [1]:
#define source and destination folders
source = 'all_data/dataset/training_set'
destination = 'all_data/training_features

# transform images in training dataset (i.e. normalized pixel values and resized pictures) to numpy arrays (smaller files, useful if uploading the set to train a model in the cloud)
for f in os.listdir(source):
    if f.find(".png") != -1:
        img = Utils.get_preprocessed_img("{}/{}".format(source, f), IMAGE_SIZE)
        file_name = f[:f.find(".png")]

        np.savez_compressed("{}/{}".format(destination, file_name), features=img)
        retrieve = np.load("{}/{}.npz".format(destination, file_name))["features"]

        assert np.array_equal(img, retrieve)

        shutil.copyfile("{}/{}.gui".format(source, file_name), "{}/{}.gui".format(destination, file_name))


NameError: name 'os' is not defined

In [ ]:
os.chdir('../')
if not os.path.exists('bin'):
    os.mkdir('bin')
os.chdir('model')

Train the model using a generator (to avoid having to fit all the data in memory)

In [ ]:
import tensorflow as tf
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

import sys

from classes.dataset.Generator import *
from classes.model.pix2code import *

np.random.seed(1234)
BATCH_SIZE = 64

dataset = Dataset()
dataset.load(input_path, generate_binary_sequences=True)
dataset.save_metadata(output_path)
dataset.voc.save(output_path)

gui_paths, img_paths = Dataset.load_paths_only(input_path)

input_shape = dataset.input_shape
output_size = dataset.output_size
steps_per_epoch = dataset.size / BATCH_SIZE
voc = Vocabulary()
voc.retrieve(output_path)

generator = Generator.data_generator(voc, gui_paths, img_paths, batch_size=BATCH_SIZE, generate_binary_sequences=True)

model.fit_generator(generator, steps_per_epoch=steps_per_epoch)